In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from collections import defaultdict

# Load the dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
X = newsgroups.data
y = newsgroups.target


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
import numpy as np
from collections import defaultdict

def fit_naive_bayes(X, y, alpha=1.0):
    class_probs = {}  # Class probabilities
    word_probs = defaultdict(lambda: defaultdict(float))  # Word probabilities for each class
    classes = set(y)  # Unique classes

    # Count the number of documents in each class
    class_counts = defaultdict(int)
    total_docs = len(y)
    for label in y:
        class_counts[label] += 1

    # Calculate class probabilities
    for label, count in class_counts.items():
        class_probs[label] = count / total_docs

    # Initialize word counts for each class
    word_counts = {label: defaultdict(int) for label in classes}

    # Count word occurrences in each class
    for i in range(total_docs):
        label = y[i]
        doc = X[i]
        for word in doc.split():
            word_counts[label][word] += 1

    # Calculate word probabilities for each class
    for label in classes:
        total_words = sum(word_counts[label].values())
        for word, count in word_counts[label].items():
            word_probs[label][word] = (count + alpha) / (total_words + alpha * len(word_counts[label]))

    return class_probs, word_probs

def predict_naive_bayes(X, class_probs, word_probs):
    predictions = []
    for doc in X:
        max_prob = float("-inf")
        predicted_class = None
        for label, class_prob in class_probs.items():
            log_prob = np.log(class_prob)
            for word in doc.split():
                if word in word_probs[label]:
                    log_prob += np.log(word_probs[label][word])
            if log_prob > max_prob:
                max_prob = log_prob
                predicted_class = label
        predictions.append(predicted_class)
    return predictions

class_probs, word_probs = fit_naive_bayes(X_train, y_train, alpha=1.0)


In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Feature extraction using CountVectorizer
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Multinomial Naive Bayes
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)


MultinomialNB()

In [6]:
# Predict using the custom Naive Bayes model
y_pred_custom = predict_naive_bayes(X_test, class_probs, word_probs)

# Predict using scikit-learn's Multinomial Naive Bayes
y_pred_sklearn = clf.predict(X_test_vec)

# Calculate accuracy for both models
accuracy_custom = accuracy_score(y_test, y_pred_custom)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

print("Accuracy using custom Naive Bayes:", accuracy_custom)
print("Accuracy using scikit-learn Multinomial Naive Bayes:", accuracy_sklearn)


Accuracy using custom Naive Bayes: 0.005305039787798408
Accuracy using scikit-learn Multinomial Naive Bayes: 0.6175066312997347


In [ ]:
#The results you've obtained indicate a significant difference in accuracy between
#the custom Naive Bayes implementation and scikit-learn's Multinomial Naive Bayes.
#The accuracy of 0.0053 for the custom Naive Bayes implementation suggests that there may be issues with the feature extraction
#or smoothing technique used, leading to very poor performance. In contrast, the accuracy of 0.6175 for scikit-learn's
#Multinomial Naive Bayes indicates much better performance on the same dataset.